# MARATONA BEHIND THE CODE 2020

## DESAFIO 7 - TNT

<hr>

## Installing Libs

In [2]:
!pip install scikit-learn --upgrade

Requirement already up-to-date: scikit-learn in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.23.2)


In [3]:
!pip install xgboost --upgrade

Requirement already up-to-date: xgboost in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.2.0)


In [4]:
!pip install imblearn --upgrade

Requirement already up-to-date: imblearn in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.0)


In [5]:
!pip install cloudant

In [6]:
from cloudant import Cloudant
u = '<username>'
p = '<pass>'
a = '<username>'
client = Cloudant(u, p, account = a, connect=True, auto_renew=True)
db = client['<your db>']

In [7]:
response = db.all_docs(limit=15000, include_docs = True)

docs = []

for r in response['rows']:
    docs.append(r['doc']['payload'])
type(docs)

list

<hr>

## Download dos conjuntos de dados em formato .csv

In [8]:
import pandas as pd

In [9]:
df_training_dataset = pd.DataFrame(data=docs)
df_training_dataset.tail()

,Açaí-Guaraná,Citrus,Estação,LAT,LONG,Maçã-Verde,Movimentação,Original_269,Original_473,Pêssego,TARGET,Tangerina,Tempo,Zero,row
12043,26,15,Luz,-23.5365,-46.6332,24,127861,19,27,9,REABASTECER,3,2019-7-30,45,11370
12044,32,13,Luz,-23.5365,-46.6332,43,125773,49,52,17,NORMAL,18,2019-10-8,16,11438
12045,20,11,Luz,-23.5365,-46.6332,9,125494,10,14,41,NORMAL,43,2019-12-15,17,11505
12046,15,43,Ana Rosa,-23.5813,-46.6383,17,66637,47,30,34,NORMAL,19,2018-10-25,49,11815
12047,25,20,São Joaquim,-23.5618,-46.6389,35,35602,10,10,25,REABASTECER,43,2018-9-30,10,12510


# Atenção!

A coluna-alvo neste desafio é a coluna ``TARGET``

<hr>

In [10]:
#IMPORTS

import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
from IPython.display import Image  
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# Fazendo as transformações e Classifier

In [11]:
id_var_name = 'id'
target_name = 'TARGET'

In [12]:
def is_imbalanced(df, target_name, tolerance):
    """
    

    Parameters
    ----------
    df : pandas.Dataframe
        df.
    target_name : string
        target_name.
    tolerance : int
        Tolerance percentage.

    Returns
    -------
    boolean
        True if is imbalanced, else False.

    """
    # Print the unique Churn values
    print(set(df[target_name]))
    # Calculate the ratio size of each churn group
    ratio_size = df.groupby([target_name]).size() / df.shape[0] * 100
    ratio_size_min = ratio_size[ratio_size.idxmin()]
    print(ratio_size)
    print(ratio_size_min)
    return (ratio_size_min <= tolerance)


imbalanced = is_imbalanced(df_training_dataset, target_name = target_name, tolerance = 30)

{'NORMAL', 'REABASTECER'}
TARGET
NORMAL         67.612882
REABASTECER    32.387118
dtype: float64
32.3871181938911


In [13]:
def split_cat_num_vars(df, id_var_name, target_name, prepare_for_ml=True, get_num=True, bf_proc_cols = []):
    if bf_proc_cols:
        df_bf_proc = df[bf_proc_cols]
        dict_rename_cols = {}
        for col in bf_proc_cols:
            dict_rename_cols[col] = col + "_bf_proc"
        df_bf_proc = df_bf_proc.rename(columns=dict_rename_cols)

    custid = [id_var_name]
    target = [target_name]
    categorical = df.nunique()[df.nunique() < 10].keys().to_list()
    if target[0] in categorical:
        categorical.remove(target[0])

    numerical = [col for col in df.columns
                 if col not in categorical + custid + target]
    
    if not prepare_for_ml:
        if get_num:
            return df[numerical]
        else:
            return df[categorical]
    else:
    
        ## One-hot encode categorical features
        # drop_first : we remove the first encoded column, as it is redundant and can be infered from the others
        df = pd.get_dummies(data = df, columns = categorical, drop_first = True)
            
        ## Scale the numerical features
        # Some ML models requires this step, because otherwise variables with larger mean and standard deviation would
        # have more influence as predictors
        
        scaler = StandardScaler()
        
        # Note: TotalCharges type = Object
        # You can use the to_numeric method in order to convert the values under the Price column into a float:
        # By setting errors=’coerce’, you’ll transform the non-numeric values into NaN.
        
        for col in df[numerical].loc[:,df[numerical].dtypes == 'object']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
        scaled_numerical = scaler.fit_transform(df[numerical])
        # Remove Nan and infinite values
        scaled_numerical = np.nan_to_num(scaled_numerical)
        
        # Build a DataFrame from scaled_numerical
        scaled_numerical = pd.DataFrame(scaled_numerical, columns=numerical)
        # scaled_numerical.fillna(scaled_numerical.mean())
        
        ## Bringing all together:
        
        # Drop non-scaled numerical columns
        df = df.drop(columns=numerical, axis=1)
        
        # Merge the non-numerical with the scaled numerical data
        df_final = df.merge(right = scaled_numerical,
                                how = 'left',
                                left_index=True,
                                right_index = True)
        if bf_proc_cols:
            df_final = df_final.merge(right = df_bf_proc,
                                    how = 'left',
                                    left_index=True,
                                    right_index = True)

    return(df_final)

In [14]:
df_training_dataset_rmcolumns = df_training_dataset.drop(columns=['Tempo', 'Estação', 'LAT', 'LONG', 'Movimentação', 'row'], inplace=False)
train_df_prepared = split_cat_num_vars(df_training_dataset_rmcolumns, id_var_name, target_name, prepare_for_ml=True, get_num=True, bf_proc_cols = [])

In [15]:
#### Split the data
def split_train_test(df, target_name, id_var_name, test_size):

    # Split X and Y into training and testing datasets
    train, test = train_test_split(df, test_size = test_size, random_state=49)
    
    # Store column names from `telcom` excluding target variable and customer ID
    cols = [col for col in df.columns if col not in id_var_name + target_name]
    
    # Extract training features
    train_X = train[cols]
    
    # Extract training target
    train_Y = train[target_name]
    
    # Extract testing features
    test_X = test[cols]
    
    # Extract testing target
    test_Y = test[target_name]
    return(train_X, train_Y, test_X, test_Y)

In [16]:
train_X, train_Y, test_X, test_Y = split_train_test(train_df_prepared, target_name=target_name, id_var_name=id_var_name, test_size=0.1)

In [17]:
train_X_res, train_Y_res= SMOTE(random_state=49, sampling_strategy=0.6).fit_resample(train_X, train_Y)

In [18]:
si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='most_frequent'
)

In [23]:
from sklearn.model_selection import StratifiedKFold

grid = Pipeline(steps=[
    ('imputer', si),
    #('Rdf', RandomForestClassifier(n_jobs=-1, max_depth=20, random_state=0))
    
    ('Rdf', OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=8, learning_rate=0.24, gamma=0.41, colsample_bytree=0.89)))
])

grid.fit(train_X_res, train_Y_res.values.ravel())

y_pred = grid.predict(test_X)

# Compute metrics
print(classification_report(test_Y, y_pred))   
print(accuracy_score(test_Y, y_pred))        


              precision    recall  f1-score   support

      NORMAL       0.92      0.87      0.90       815
 REABASTECER       0.76      0.84      0.80       390

    accuracy                           0.86      1205
   macro avg       0.84      0.86      0.85      1205
weighted avg       0.87      0.86      0.86      1205

0.862240663900415


### Analisando a qualidade do modelo através da matriz de confusão

<hr>

## Scoring dos dados necessários para entregar a solução

Como entrega da sua solução, esperamos os resultados classificados no seguinte dataset chamado "to_be_scored.csv":

### Download da "folha de respostas"

In [24]:
!wget --no-check-certificate --content-disposition https://gitlab.com/JoaoPedroPP/datasets/-/raw/master/ntn/to_be_scored.csv
df_to_be_scored = pd.read_csv(r'to_be_scored.csv')

--2020-09-10 21:01:49--  https://gitlab.com/JoaoPedroPP/datasets/-/raw/master/ntn/to_be_scored.csv
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘to_be_scored.csv.2’

    [ <=>                                   ] 70,291      --.-K/s   in 0.001s  

2020-09-10 21:01:49 (51.6 MB/s) - ‘to_be_scored.csv.2’ saved [70291]



<hr>

# Atenção!

# Para poder aplicar seu modelo e classificar a folha de respostas, você precisa primeiro aplicar as mesmas transformações com colunas que você aplicou no dataset de treino.

# Não remova ou adicione linhas na folha de respostas. 

# Não altere a ordem das linhas na folha de respostas.

# Ao final, as 1000 entradas devem estar classificadas, com os valores previstos em uma coluna chamada "target"

<hr>

Na célula abaixo, repetimos rapidamente os mesmos passos de pré-processamento usados no exemplo dado com árvore de decisão

In [25]:
df_to_be_scored_rmcolumns = df_to_be_scored.drop(columns=['Tempo', 'Estação', 'LAT', 'LONG', 'Movimentação'], inplace=False)

df_to_be_scored_prepared = split_cat_num_vars(df_to_be_scored_rmcolumns, id_var_name, target_name, prepare_for_ml=True, get_num=True, bf_proc_cols = [])

<hr>

Pode ser verificado abaixo que as colunas da folha de resposta agora são idênticas às que foram usadas para treinar o modelo:

# Atenção

Para todas colunas que não existirem no "df_to_be_scored", você pode usar a técnica abaixo para adicioná-las:

In [26]:
y_pred = grid.predict(df_to_be_scored_prepared)
df_to_be_scored["TARGET"] = y_pred

### Salvando a folha de respostas como um arquivo .csv para ser submetido

In [ ]:
project.save_data(file_name="results.csv", data=df_to_be_scored.to_csv(index=False))

# Atenção

# A execução da célula acima irá criar um novo "data asset" no seu projeto no Watson Studio. Você precisará realizar o download deste arquivo juntamente com este notebook e criar um arquivo zip com os arquivos **results.csv** e **notebook.ipynb** para submissão. (os arquivos devem estar nomeados desta forma)

<hr>

## Parabéns!

Se você já está satisfeito com a sua solução, vá até a página abaixo e envie os arquivos necessários para submissão.

# https://tnt.maratona.dev
